In [1]:
import hashlib
import os
import tarfile
import zipfile
import requests
import numpy as np
import pandas as pd
import torch
from torch import nn
from d2l import torch as d2l
import unicodedata
from collections import Counter

In [2]:
# 加载数据集
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
print(f'训练集：{train_data.shape}')
print(f'测试集：{test_data.shape}')

训练集：(47439, 41)
测试集：(31626, 40)


In [3]:
# Drop 标签
irrelevant_columns = ['Address', 'Summary', 'State', 'Heating', 'Cooling', 'Elementary School',
                      'Middle School', 'High School', 'Flooring', 'Cooling features', 'Parking features',
                      'Appliances included', 'Laundry features', 'Last Sold On', 'Listed On']

In [4]:
# 稀疏类别标签
comb_columns = ['Type', 'Region', 'Heating features', 'City']

In [5]:
# 特殊的数据标签
"""
数字和文本混合, 
文本内容描述的是拥有的若干卧室，并用,隔开，于是可以将文本的逗号数+1，得到卧室的数量
统计逗号的数量，可以使用collections包里的Counter类，
由于列表中的部分数字是以字符串形式存放的，在此之前判断一下内容是否为数字
"""
List1 = ['Bedrooms', 'Parking']

In [6]:
# 移除不相关特征
train_features = train_data.drop(columns=['Sold Price'])
train_features = train_features.drop(columns=irrelevant_columns)
test_features = test_data.drop(columns=irrelevant_columns)

In [7]:
# 拼接两个数据集，这里就会让 Id 标签消失
all_features = pd.concat((train_features.iloc[:, 1:],test_features.iloc[:, 1:]))

In [8]:
# List1 处理（特殊标签处理）
def count_comma(s):
    """ 计算字符串中逗号(',')出现的次数
    
    该函数使用collections.Counter统计字符串中每个字符的出现次数，
    然后返回逗号字符','的出现次数
    
    参数:
        s (str): 需要统计逗号数量的输入字符串
        
    返回:
        int: 字符串中逗号出现的次数
        
    示例:
        >>> count_comma("1,2,3")
        2
        >>> count_comma("Single")
        0
    """
    dic = Counter(s)  # 创建字符计数器，统计字符串中每个字符的出现次数
    return dic[',']  # 返回逗号字符','的计数结果

In [11]:
# test Counter() Function
txt = '12333211abbbccbca测试测试测测试'
test_dic = Counter(txt)
test_dic

Counter({'1': 3, '2': 2, '3': 3, 'a': 2, 'b': 4, 'c': 3, '测': 4, '试': 3})

In [13]:
def is_number(s):
    """ 检查输入字符串是否能转换为数值类型（整数或浮点数）
    
    该函数尝试通过两种方式验证字符串是否为有效数字：
    1. 尝试转换为浮点数 (float)
    2. 尝试解析为Unicode数字字符 (如中文数字"三")
    
    参数:
        s (str): 要检查的输入字符串
        
    返回:
        bool: 如果字符串可转换为数字则返回True，否则返回False
        
    示例:
        >>> is_number("123")
        True
        >>> is_number("12.34")
        True
        >>> is_number("三")  # 中文数字"三"
        True
        >>> is_number("abc")
        False
    """
    # 尝试方法 1: 转换为浮点数
    try:
        float(s)
        return True
    except ValueError:
        pass  # 不做处理，继续尝试下一种方法

    # 尝试方法 2: 解析为 Unicode 数字字符
    try:
        unicodedata.numeric(s)
        return True
    except (TypeError, ValueError):
        pass

    # 所有转换尝试都失败
    return False  # 返回 False 表示不是有效数字

In [18]:
# test is_number() function
is_number('十')

True

In [20]:
def comma_to_number(features,list):
    """
    将指定列中非数值格式的字符串转换为数值
    
    该函数专门处理包含逗号分隔值的特殊格式列（如 "1,2,3"），
    将其转换为表示条目数量的数值（逗号数量 + 1）。
    
    参数:
        features (pd.DataFrame): 包含需要处理特征的数据框
        list (list): 需要处理的列名列表（如 ['Bedrooms', 'Parking']）
        
    返回:
        pd.DataFrame: 处理后的数据框副本（索引已重置）
        
    处理流程:
        1. 创建数据的安全拷贝（避免修改原始数据）
        2. 重置索引（确保后续定位操作准确）
        3. 遍历指定的每一列
        4. 对每列中的每个元素:
            - 检查是否为数值
            - 如果不是数值: 计算逗号数量并加1作为新值
        5. 将整列转换为浮点数类型（确保数值一致性）
        
    示例:
        输入: "1,2,3" → 处理: 逗号数=2 → 新值=3 (2+1)
        输入: "Single" → 处理: 逗号数=0 → 新值=1 (0+1)
        输入: "3" → 处理: 保留原值3 → 转换为3.0
    """
    # 重置索引并丢弃原索引，避免索引不连续导致的定位错误
    X = features.reset_index(drop=True).copy()

    # 遍历需要处理的列
    for cols in list:
        # 遍历当前列的每个元素（使用enumerate同时获取索引i和值item）
        for i,item in enumerate(X[cols]):
            # 检查当前元素是否为数值
            if not is_number(item):
                X.loc[i, cols] = count_comma(item)+1
        # 处理完当前列的所有元素后，将整列转换为浮点数类型
        X[cols] = X[cols].astype('float')
    return X

In [23]:
# test comma_to_number()

# 创建测试数据框
test_data = {
    'Bedrooms': ['1', '2,3', '1,2,3', 'Single', 'Studio', '3', ''],
    'Parking': ['Garage', '1,2', 'None', '3', '1,2,3', 'Carport', 'Street Parking'],
    'Other_Column': [10, 20, 30, 40, 50, 60, 70]  # 不应被处理的列
}

test_df = pd.DataFrame(test_data)

# 定义需要处理的列
columns_to_process = ['Bedrooms', 'Parking']

# 应用转换函数
result = comma_to_number(test_df, columns_to_process)

print(f"原始数据: {test_data}")
print(f'处理后数据：{result}')

原始数据: {'Bedrooms': ['1', '2,3', '1,2,3', 'Single', 'Studio', '3', ''], 'Parking': ['Garage', '1,2', 'None', '3', '1,2,3', 'Carport', 'Street Parking'], 'Other_Column': [10, 20, 30, 40, 50, 60, 70]}
处理后数据：   Bedrooms  Parking  Other_Column
0       1.0      1.0            10
1       2.0      2.0            20
2       3.0      1.0            30
3       1.0      3.0            40
4       1.0      3.0            50
5       3.0      1.0            60
6       1.0      1.0            70


In [24]:
# 执行对文字数字混合特殊列的处理
all_features = comma_to_number(all_features, List1)

In [30]:
def combine_rare_categories(df, columns, threshold=1):
    """
    将指定列中的稀疏类别合并为'Other'类别
    
    该函数用于处理分类特征中的稀疏类别（出现频率低的类别），
    将它们统一合并为'Other'类别，从而减少特征维度并增强模型稳定性。
    
    参数:
        df (pd.DataFrame): 需要处理的数据框
        columns (list): 需要处理的列名列表
        threshold (float): 类别频率阈值（百分比），默认为1%
            - 出现频率低于此阈值的类别将被合并
    
    返回:
        pd.DataFrame: 处理后的数据框（原地修改）
    
    处理逻辑:
        1. 遍历指定的每一列
        2. 如果列存在于数据框中:
            a. 计算每个类别的相对频率（百分比）
            b. 识别频率低于阈值的稀疏类别
            c. 将这些稀疏类别的值替换为'Other'
    
    示例:
        假设某列有100个样本:
          - 类别A出现98次（98%）
          - 类别B出现1次（1%）
          - 类别C出现1次（1%）
        设置threshold=1时，类别B和C将被合并为'Other'
    """
    # 遍历所有指定列
    for col in columns:
        # 检查当前列是否存在于数据框中
        if col in df.columns:  
            # 计算当前列每个类别的相对频率（转换为百分比）
            value_counts = df[col].value_counts(normalize=True) * 100

            # 使用布尔索引筛选出符合条件的类别及其频率
            rare_categories = value_counts[value_counts < threshold].index

            # 应用替换：将稀疏类别替换为'Other'
            df[col] = df[col].apply(lambda x: 'Other' if x in rare_categories else x)  

    # 返回处理后的数据框（原地修改）
    return df

In [31]:
# test combine_rare_categories()
# 创建测试数据
test_data = pd.DataFrame({
    'City': ['NY']*88 + ['LA'] + ['SF'] + ['CH']*3 + ['MI']*2 + ['PHX']*5,
    'Type': ['House']*60 + ['Apartment']*35 + ['Condo']*4 + ['Loft']*1
})

print("处理前:")
print("City列分布:\n", test_data['City'].value_counts(normalize=True)*100)
print("\nType列分布:\n", test_data['Type'].value_counts(normalize=True)*100)

# 应用函数（设置threshold=3）
processed_data = combine_rare_categories(test_data, ['City', 'Type'], threshold=3)

print("\n处理后:")
print("City列分布:\n", processed_data['City'].value_counts(normalize=True)*100)
print("\nType列分布:\n", processed_data['Type'].value_counts(normalize=True)*100)

处理前:
City列分布:
 NY     88.0
PHX     5.0
CH      3.0
MI      2.0
SF      1.0
LA      1.0
Name: City, dtype: float64

Type列分布:
 House        60.0
Apartment    35.0
Condo         4.0
Loft          1.0
Name: Type, dtype: float64

处理后:
City列分布:
 NY       88.0
PHX       5.0
Other     4.0
CH        3.0
Name: City, dtype: float64

Type列分布:
 House        60.0
Apartment    35.0
Condo         4.0
Other         1.0
Name: Type, dtype: float64


In [33]:
# 合并稀疏项
df_combined = combine_rare_categories(all_features, comb_columns, threshold=1)

# 查看合并后的数据集
print(df_combined.head())

           Type  Year built  Parking     Lot  Bedrooms  Bathrooms  \
0  SingleFamily      1969.0      3.0     1.0       4.0        0.0   
1  SingleFamily      1926.0      2.0  4047.0       3.0        2.0   
2  SingleFamily      1958.0      1.0  9147.0       2.0        3.0   
3  SingleFamily      1947.0      3.0     NaN       5.0        3.0   
4    VacantLand         NaN      1.0     NaN       NaN        NaN   

   Full bathrooms  Total interior livable area  Total spaces  Garage spaces  \
0             NaN                          1.0           0.0            0.0   
1             2.0                        872.0           1.0            1.0   
2             1.0                       1152.0           0.0            0.0   
3             3.0                       2612.0           0.0            0.0   
4             NaN                          NaN           NaN            NaN   

   ... Middle School Distance  High School Score  High School Distance  \
0  ...                    NaN       

In [34]:
# 自此我们保存的 all_features 进行了 drop 不需要的项，以及合并稀疏矩阵，对 bedroom 和 parking 进行了数字化处理
all_features.to_csv('all_features.csv', index=False)